# UNet Segmentation of Optic Disc and Optic Cup

In [10]:
import os
import numpy as np
import torch
from PIL import Image
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from GlaucomaDataset import GlaucomaDataset

In [2]:
origa_path = os.path.join("..", "data", "ORIGA")
images_path = os.path.join(origa_path, "Images_Square")
masks_path = os.path.join(origa_path, "Masks_Square")

img_filenames = sorted(os.listdir(images_path))
mask_filenames = sorted(os.listdir(masks_path))

In [3]:
# Split into train, validation, and test sets (70, 15, 15)
train_imgs, temp_imgs, train_masks, temp_masks = train_test_split(
    img_filenames, mask_filenames, test_size=0.3, random_state=42)

val_imgs, test_imgs, val_masks, test_masks = train_test_split(
    temp_imgs, temp_masks, test_size=0.5, random_state=42)

In [9]:
# TODO: Set parameters (learning rate, batch size, epochs, etc.)
lr = 1e-4
batch_size = 8
n_workers = 4
epochs = 40

In [11]:
# Load data
train_set = GlaucomaDataset(images_path, masks_path, train_imgs, train_masks)
val_set = GlaucomaDataset(images_path, masks_path, val_imgs, val_masks)
test_set = GlaucomaDataset(images_path, masks_path, test_imgs, test_masks)

train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=n_workers, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=n_workers, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=n_workers, shuffle=True) # need to load test?

In [10]:
# TODO: Initialize model (device, model, loss, optimizer)


In [ ]:
# TODO: Define metrics (before or after training?)

In [11]:
# TODO: Train model 

In [ ]:
# To consider: adding grayscale / normalization (either for images or batches), adding a remove nerves function
# Implement early stopping?

In [ ]:
# TODO: Tune model (cv for lr, epochs, batch size, etc.)